In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats

In [ ]:
import random
random.seed(13)

### Import data

In [ ]:
# maybe use np.where to reduce the data imported

Read

In [ ]:
# code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [ ]:
# need to import this file in slightly different way because of different delimiters
clinical = pd.read_csv('data/covid19_clinical_sample.txt',sep='\t|,', engine = 'python')

In [ ]:
clinical_old = pd.read_csv('data/tpp_gp_clinical_subset.csv')

In [ ]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [ ]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv') 

In [ ]:
# need to import this file in slightly different way because of different delimiters
scripts = pd.read_csv('data/covid19_prescriptions_sample.txt', sep='\t|,', engine = 'python')

In [ ]:
scripts_old = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

In [ ]:
mortality = pd.read_csv('data/patients_data_death_sample.csv')

In [ ]:
mortality_coding = pd.read_csv('data/patients_data_death_decoding.csv')

have a look at the data

In [ ]:
# code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

In [ ]:
clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

In [ ]:
clinical_old

In [ ]:
ic10
# description of ICD20 codes

In [ ]:
GPic10
#translates CTV3 codes (read_code) to ICD10 (icd10_code)

In [ ]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 
# needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

In [ ]:
scripts_old

In [ ]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

### Preparing the data

#### Adding column with IC10 categories to clinical dataset for simpler analysis later on

In [ ]:
# create a reduced dataset for merging with the clinical dataset 
GPic10_red =  GPic10[['read_code', 'icd10_code']]
# rename to merge ont he ctv3 column with the clinical dataset
GPic10_red = GPic10_red.rename(columns = {'read_code':'code'})
GPic10_red.head()

In [ ]:
clinical_ic10 = pd.merge(clinical, GPic10_red, how='inner', on='code')
clinical_ic10

#### Data Cleaning

In [ ]:
# drop unnecessary columns
clinical_ic10.drop(columns = ['code_type', 'value', 'Unnamed: 0'], inplace = True)
clinical_ic10

In [ ]:
pd.isnull(clinical_ic10).sum()
# no NaN

In [ ]:
pd.isnull(ic10).sum()

In [ ]:
# remove duplicate entries in clinical dataset 
# (for other datasets, it should not be a problem)
clinical_ic10.drop_duplicates(inplace = True) 

# remove NaN in selected column
ic10.dropna(subset=['DESCRIPTION'], inplace = True)

In [ ]:
pd.isnull(ic10).sum()

In [ ]:
clinical_ic10

In [ ]:
type(('O','P','S','T','U','V','W','X','Y','Z'))

In [ ]:
# remove not considered clinical events in the clinical dataset (e.g. external factors)

""""
Monica's suggestions: 
remove O-Z (pregnancy, congenital, others, COVID-19, external causes, factors)
maybe keep Q (congenital) as underlying disease, keep all others

Renée:
I would also keep R in there: "In general, categories in this chapter include the less well-defined conditions and symptoms that, without the necessary study of the case to establish a final diagnosis, point perhaps equally to two or more diseases or to two or more systems of the body.

Decision:
O-Z remove, but leave R as adverse event & Q as underlying disease.

"""

# tuple of categories we do not consider as adverse event or underlying disease:
# the ones starting with O-Z (except Q and R)
exclude = ('O','P','S','T','U','V','W','X','Y','Z')

# broadcasting the clinical dataset for all icd codes except those we exclude
clinical_ic10 = clinical_ic10[~clinical_ic10['icd10_code'].astype(str).str.startswith(exclude)]
clinical_ic10

#### Rename columns of mortality dataframe

In [ ]:
scripts_old

In [ ]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

### Preparing the data

#### Adding column with IC10 categories to clinical dataset for simpler analysis later on

In [ ]:
# create a reduced dataset for merging with the clinical dataset 
GPic10_red =  GPic10[['read_code', 'icd10_code']]
# rename to merge ont he ctv3 column with the clinical dataset
GPic10_red = GPic10_red.rename(columns = {'read_code':'code'})
GPic10_red.head()

In [ ]:
clinical_ic10 = pd.merge(clinical, GPic10_red, how='inner', on='code')
clinical_ic10

#### Data Cleaning

In [ ]:
# drop unnecessary columns
clinical_ic10.drop(columns = ['code_type', 'value', 'Unnamed: 0'], inplace = True)
clinical_ic10

In [ ]:
pd.isnull(clinical_ic10).sum()
# no NaN

In [ ]:
pd.isnull(ic10).sum()

In [ ]:
# remove duplicate entries in clinical dataset 
# (for other datasets, it should not be a problem)
clinical_ic10.drop_duplicates(inplace = True) 

# remove NaN in selected column
ic10.dropna(subset=['DESCRIPTION'], inplace = True)

In [ ]:
pd.isnull(ic10).sum()

In [ ]:
clinical_ic10

In [ ]:
type(('O','P','S','T','U','V','W','X','Y','Z'))

In [ ]:
# remove not considered clinical events in the clinical dataset (e.g. external factors)

""""
Monica's suggestions: 
remove O-Z (pregnancy, congenital, others, COVID-19, external causes, factors)
maybe keep Q (congenital) as underlying disease, keep all others

Renée:
I would also keep R in there: "In general, categories in this chapter include the less well-defined conditions and symptoms that, without the necessary study of the case to establish a final diagnosis, point perhaps equally to two or more diseases or to two or more systems of the body.

Decision:
O-Z remove, but leave R as adverse event & Q as underlying disease.

"""

# tuple of categories we do not consider as adverse event or underlying disease:
# the ones starting with O-Z (except Q and R)
exclude = ('O','P','S','T','U','V','W','X','Y','Z')

# broadcasting the clinical dataset for all icd codes except those we exclude
clinical_ic10 = clinical_ic10[~clinical_ic10['icd10_code'].str.startswith(exclude)]
clinical_ic10

#### Rename columns of mortality dataframe

In [ ]:
mortality.head()

,Unnamed: 0,eid,31-0.0,34-0.0,52-0.0,53-0.0,21001-0.0,21022-0.0,40000-0.0,40001-0.0,40001-1.0,40007-0.0
0,334517,4345610,1.0,1940.0,2.0,2009-09-17,28.4091,69.0,NaN,NaN,NaN,NaN
1,297000,3970369,1.0,1949.0,12.0,2008-04-18,26.1106,58.0,NaN,NaN,NaN,NaN
2,260822,3608558,1.0,1943.0,2.0,2010-02-26,27.4614,67.0,NaN,NaN,NaN,NaN
3,245361,3453904,0.0,1957.0,8.0,2010-03-19,29.3199,52.0,NaN,NaN,NaN,NaN
4,25616,1256219,1.0,1959.0,7.0,2008-12-20,32.7859,49.0,NaN,NaN,NaN,NaN


In [32]:
mortality_coding

,Unnamed: 0,description
0,eid,PatientID
1,21022-0.0,AgeRecruit
2,53-0.0,DateRecruit
3,34-0.0,YearBirth
4,52-0.0,MonthBirth
5,40007-0.0,AgeDeath
6,40000-0.0,DateDeath
7,31-0.0,Sex
8,21001-0.0,BMI
9,40001-0.0,cause of death 1


In [ ]:
# make a dictionary out of the mortality_coding dataframe 
dict_columns = {}
for index, value in enumerate(mortality_coding['Unnamed: 0']):
  dict_columns[value] = mortality_coding.loc[index, "description"]

In [ ]:
mortality.rename(columns = dict_columns, inplace = True) # columns = a dictionary will automatically rename the columns
mortality['DateDeath'].isna().sum()
# mortality['DateDeath'].value_counts()


In [ ]:
mortality

## Find who is vaccinated

The meaning of the SNOMED codes ('dmd_code' in the 'scripts' dataset, where medications that the patients got are listed) were found on: https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

Each vaccine has a specific code.

In [ ]:
# broadcasting to find individuals with AstraZEneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ].head()

In [ ]:
# getting IDs of individuals with any corona vaccine first (compare vaccinates vs. not)
# may look at effects of specific vaccines later
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

for dmd_code in dict_vac:
    mask_vac = scripts['dmd_code']== dmd_code
    print(dict_vac[dmd_code], ':',len(scripts[mask_vac]))

In [ ]:
# mask for any vaccine
mask_vac = scripts['dmd_code'].isin(dict_vac)

# number of vaccinated people
print('Number of vaccinated people:', len(scripts[mask_vac]))

In [ ]:
# finding eids of vaccinated people in the dataset
id_vac = scripts[mask_vac]['eid'] 
id_vac

In [ ]:
# finding events of the vaccinated people, but only events before vaccination found 
# -> may need a more recent dataset
# need to separate clinical events before & after vaccine
mask_vac_clin = clinical_ic10['eid'].isin(id_vac)
clinical_ic10[mask_vac_clin] 

# categorize event as before or after COVID-19 vaccination / random date 

### make 'event_date' in 'clinical' be interpreted as a date

In [ ]:
clinical_ic10.head(10)

In [ ]:
from datetime import datetime, timedelta

### only to understand the function, can be deleted later

In [ ]:
clinical_ic10['event_dt'].unique()
# find out if the other dates are e.g. 06/01/2020 or 6/1/2020

In [ ]:
example_date = clinical_ic10['event_dt'].unique()[0]

In [ ]:
example_date_new = datetime.strptime(example_date, '%d/%m/%Y')
example_date_new #2020

In [ ]:
example_date_2 = clinical_ic10['event_dt'].unique()[1]
example_date_2 

In [ ]:
example_date_2_new = datetime.strptime(example_date_2, '%d/%m/%Y')
example_date_2_new #2019

In [ ]:
# compare the 2 dates
example_date_new > example_date_2_new # 2020 > 2019 --> True
# recent > older --> True

### convert the column into datetime object

In [ ]:
# convert entire event_dt in datetime format
clinical_ic10['event_date_format'] = pd.to_datetime(clinical_ic10['event_dt'])
clinical_ic10

In [ ]:
# checking the date range
clinical_ic10.sort_values(by = 'event_date_format', ascending = False)

In [ ]:
# only to understand the function, can be deleted later!!
# compare 2 dates (first row and last row) from the new column 

print(clinical_ic10['event_date_format'].iloc[-1]) # 2019
print(clinical_ic10['event_date_format'].iloc[0]) # 2020
clinical_ic10['event_date_format'].iloc[0] > clinical_ic10['event_date_format'].iloc[-1]

### subset 'scripts' to only those patients (eid) that got the COVID-19 vaccine and drop 'Unnamed: 0' column

In [ ]:
scripts.drop(columns = ['Unnamed: 0'], inplace = True)
mask_scripts_vac = scripts['dmd_code'].isin(dict_vac)
scripts_vaccines_only = scripts[mask_scripts_vac] 

len(scripts[mask_scripts_vac]) # corresponds to finding above

In [ ]:
scripts_vaccines_only.head() # contains eid and date of vaccination

In [ ]:
# gives warning (same as before, should maybe be checked!)
scripts_vaccines_only['issue_date_format'] = pd.to_datetime(scripts_vaccines_only['issue_date'])


In [ ]:
scripts_vaccines_only

In [ ]:
clinical_ic10.head()
# length: 4156
#len(clinical_ic10['eid'].unique()) # some eid are in there multiple times (of course)

### adding deaths to clinical_ic10 df so they can be treated as adverse events in first analysis (?)

In [ ]:
# subset mortality df to the eids with a deathdate that is not NaN
mortality_sub = mortality.dropna(subset=['DateDeath'])
# drop all the unnecessary columns (also drop age and sex, perhaps no time to analyse this), keep eid, deathdate, cause
mortality_sub.drop(columns = ['Unnamed: 0', 'Sex', 'YearBirth', 'MonthBirth', 'DateRecruit', 'BMI', 'AgeDeath'], inplace = True)
# rename column 'PatientID' to 'eid' and 'DateDeath' to 'issue_date_format'
mortality_sub.rename(columns = {'PatientID':'eid', 'DateDeath':'event_date_format'}, inplace = True)

In [ ]:
# add column 'icd10_code' to mortality, filled with string 'death'
mortality_sub['icd10_code'] = pd.Series(["death" for x in range(len(mortality.index))])


In [ ]:
# make 'DateDeath' datetime type
mortality_sub['event_date_format'] = pd.to_datetime(mortality_sub['event_date_format'])


In [ ]:
mortality_sub

In [ ]:
# now join it to the clinical_vac
clinical_ic10= pd.concat([clinical_ic10, mortality_sub], join = 'outer')


In [ ]:
clinical_ic10

### add a column with the date of vaccination to clinical_ic10 and drop event_dt and issue_date

In [ ]:
# merge clinical_ic10 and scripts_vaccines_only to add the issue_date (date of vaccine) to the df
clinical_vac = pd.merge(clinical_ic10, scripts_vaccines_only, how ='outer', on ='eid')

In [ ]:
# convert issue date to date format
# clinical_vac['issue_date_format'] = pd.to_datetime(clinical_vac['issue_date'])
# renée: I guess it's safer, since the dtype is 'datetime64' in the new column and 'object' in the old one:
# print(clinical_vac['issue_date']) 
# print(clinical_vac['issue_date_format'])

In [ ]:
# drop event_dt and issue_date
clinical_vac.drop(columns = ['event_dt', 'issue_date'], inplace = True)

In [ ]:
len(clinical_vac) 
# total amounts of events: vaccinations and medical events from clinical_ic10

In [ ]:
clinical_vac

In [ ]:
clinical_vac['issue_date_format'].isna().sum()

### add column in clinical_vac that says whether a patient is vaccinated or not

In [ ]:
mask_vaccinated = clinical_vac['eid'].isin(id_vac)
clinical_vac['vaccinated'] = mask_vaccinated

# check how many events of vaccinated people there are
clinical_vac['vaccinated'].value_counts()

# clinical_ic10['eid'].loc[clinical_ic10['vaccinated']==True]

In [ ]:
clinical_vac 
# 133029 events
# rows with event_dt=NaN: vaccinated eids without medical events -> maybe rename it

### add random issue date to events of unvaccinated patients to 'issue_date_format'  in clinical_vac 

In [ ]:
# make list from 'issue_date' in scripts_vaccines_only
issue_dates = scripts_vaccines_only['issue_date_format'].tolist()
issue_dates

In [ ]:
# 'eid' of unvaccinated people
id_nvac = clinical_vac[clinical_vac['vaccinated']==False]['eid'].unique()
len(id_nvac)

In [ ]:
# create list of new dates
issue_dt_random = random.choices(issue_dates, k = len(id_nvac))
len(issue_dt_random)

In [ ]:
# make dictionary with random issue dates and unvaccinated eid
dict_nvac = dict(zip(id_nvac,issue_dt_random))
len(dict_nvac)

In [ ]:
# fill na with mapping from the dictionary
clinical_vac['issue_date_format'] = clinical_vac['issue_date_format'].fillna(clinical_vac['eid'].map(dict_nvac))

In [ ]:
clinical_vac['issue_date_format'][clinical_vac['vaccinated']==True].unique()

In [ ]:
clinical_vac['issue_date_format'][clinical_vac['vaccinated']==False].unique()

In [ ]:
clinical_vac[clinical_vac['vaccinated'] == False]

In [ ]:
clinical_vac['issue_date_format'].isna().sum()

### add a column where it says if the event (event_date_format) was before or after the vaccination (issue_date)

In [ ]:
before_after_vaccine = []
for i in range (len(clinical_vac)):
    if clinical_vac['event_date_format'].iloc[i] >= clinical_vac['issue_date_format'].iloc[i]:
        before_after_vaccine.append('after')
    else:
        before_after_vaccine.append('before')
clinical_vac['before_after_vaccine'] = before_after_vaccine

In [ ]:
clinical_vac

In [ ]:
# checking how many events before and after
clinical_vac['before_after_vaccine'].value_counts()

# if event_date_format is NaT, is says before in 'before_after_vaccine'

# Analysing clinical events

IC10 codes are structured. The following website was used for looking up the meanings of the categories (e.g. respiratory diseases):
https://www.icd10data.com/ICD10CM/Codes

In [ ]:
ic10.head()

the GP_ic10 has the read code which is the CTV3. 
(However, when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc)

In [ ]:
# WILL PROBABLY NOT USE THIS

# just to see if corresponding CTV3 to IC10 works

# broadcasting for vaccinated comorbidities(CTV3) in ICD10
mask_vac_ic10 = GPic10['read_code'].isin(clinical_ic10[mask_vac_clin]['code']) 
GPic10[mask_vac_ic10]

# mask_vac: people that got any vaccine
# search for patients that got the vaccine in GPic10
# and look what clinical events they had via the read_code (e.g. XE01qn)


if time: analyse specific diseases such as myocarditis etc.

In [ ]:
clinical_ic10

In [ ]:
ic10.head(3)

In [ ]:
# mask for getting codes related to myocarditis
mask_myocarditis = ic10['DESCRIPTION'].str.contains('myocarditis', case = False)
ic10[mask_myocarditis]

In [ ]:
# getting data with myocarditis

# the alternative code 'ALT_CODE' used here, as this was used in the lookup table GPic10
myocarditis = ic10[mask_myocarditis]['ALT_CODE']

clinical_ic10[clinical_ic10['icd10_code'].isin(myocarditis)]
# nobody had myocarditis in this restricted data

In [ ]:
## ishaemic heart disease
mask_ihd = ic10['DESCRIPTION'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = ic10[mask_ihd]['ALT_CODE']
clinical_ic10[clinical_ic10['icd10_code'].isin(ischaemicHD)]

several comorbidities

In [ ]:
# getting people with specific diseases

# new column for filling with selected disease
clinical_ic10['selected disease'] = np.nan 

# list of specific diseases, to be decided on
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] 

for disease in diseases_selec:
    
    # broadcasting for selected disease
    mask = ic10['DESCRIPTION'].str.contains(disease, case = False) 
    # broadcasting for selected disease
    sub_disease = clinical_ic10[clinical_ic10['icd10_code'].isin(ic10[mask]['ALT_CODE'])]
    
    for i in sub_disease.index:
        # fill dataset with selected disease label
        clinical_ic10.loc[i,'selected disease'] = disease 

In [ ]:
# counting how many people in the dataset have a specific disease
clinical_ic10['selected disease'].value_counts()

# getting people w. different categories of underlying diseases

# Question 1: What is the effect on adverse events?
## 1. How common are "adverse events"? (i.e. how many percent of vaccinated people have registered medical events after the vaccine) compare vaccinated & nonvaccinated
## 2. What about people with medical history?
## 3. Are they more common in people with (specific) underlying medical conditions? i.e.: 
* plot percentage of AEs with & without underlying conditions
* plot percentages for specific underlying conditions (not considering if several conditions at once, plot for each category then), for vaccinated and non-vaccinated people
* (optional: just for vaccinated people: plot what types of AEs (in IC10 categories) are common.

## 1. How common are "adverse events"? (i.e. how many percent of vaccinated people have registered medical events after the vaccine) compare vaccinated & nonvaccinated

In [ ]:
# initialize column & index names
dict_vacORnot = {'vaccinated':1, 'not vaccinated':0} #vaccination status
ls_AE = ['AE', 'no AE'] # adverse events

Get number of adverse events (defined medical events w. ICD10 code after the (artificial) vaccination date)

In [ ]:
def incidence(df_event, id_vac, id_nvac):
    """
    Calculates the number of AEs per thousand individuals in a given population.
    
    Input:
    - df_event: a dataframe containing all events considered in the analysis (e.g. AEs)
    - id_vac: series containing ID's of each individual (with each individual listed once) within the given population.
    - id_nvac: same as id_vac but with all non-vaccinated individuals
    
    
    """
    
    # initialize df for putting in the values
    df_sum = pd.DataFrame(columns = ['Number of AEs', 'Number of individuals', 'Number of AEs per thousand individuals'], index = dict_vacORnot)
    
    for v in dict_vacORnot: # loop through vaccinated vs. not
        
        # put in the number of events (AEs) by subsetting the df_event into vaccinated/not vaccinated
        df_sum['Number of AEs'][v] = len(df_event[df_event['vaccinated'] == dict_vacORnot[v]])
    
        # if we have no event (happens in the subset), add 0.5 for testing the plot and analysis further below
        # -> still visible in the output that it was added
        if df_sum['Number of AEs'][v]==0:
            df_sum['Number of AEs'][v]=0.5
    
    # put number of vaccinated/not vaccinated people in the population into the summary df
    df_sum['Number of individuals']['vaccinated'] = len(id_vac) # vaccinated
    df_sum['Number of individuals']['not vaccinated'] = len(id_nvac) # not vaccinated
    
    # calculate AEs/individual
    df_sum['Number of AEs per thousand individuals'] = df_sum['Number of AEs']/df_sum['Number of individuals']*1000
    
    return df_sum

In [ ]:
# Make a df with only AEs (defined medical events w. ICD10 code after the (artificial) vaccination date)
# reminder: leave R as adverse event & Q as underlying disease.

# subset of clinical dataset for adverse events (=after vaccine)
df_AEs = clinical_vac[clinical_vac['before_after_vaccine']== 'after']

# remove Q (congenital diseases), which are not considered as adverse events
df_AEs = df_AEs[~df_AEs['icd10_code'].str.startswith('Q')]

In [ ]:
# get number of people of the chosen population
# list of all IDs from the clinical dataset
id_clin = clinical['eid'].unique()
# list of all IDs from the scripts dataset
id_scr = scripts['eid'].unique()

# concatenate both lists to get all IDs in one list
id_all = np.concatenate((id_clin, id_scr))

# leave only unique IDs in the list to get the number of IDs using len()
id_all = np.unique(id_all)

# get array with IDs without vaccine by deleting the IDs with vaccine out of the array with all IDs
id_nvac = np.delete(id_all, np.isin(id_all,id_vac))

In [ ]:
# use the function to create the summary table
df_sum_all = incidence(df_AEs, id_vac, id_nvac)
df_sum_all

Plot how oftern AEs occur in vaccinated vs. not vaccinated people for all individuals in the dataset.

In [ ]:
plt.figure(figsize = (5,5))
plt.bar(df_sum_all.index, df_sum_all['Number of AEs per thousand individuals'])
plt.ylabel('Number of AEs per thousand individuals')
plt.title('Adverse events in the whole population of the dataset')

Make contingency table (not using pd.crosstab to avoid another huge dataset with all individuals and because we need to count unique IDs)

In [ ]:
def contingency(df_event, id_vac, id_nvac, columns = ls_AE, index = dict_vacORnot):
    """
    Makes a 2x2 contingency table, which can be used for the chi^2 test and calculating the risk ratio.
    
    Input:
    - df_event: a dataframe containing all events considered in the analysis (e.g. AEs)
    - id_vac: series containing ID's of each individual (with each individual listed once) within the given population.
    - id_nvac: same as id_vac but with all non-vaccinated individuals
    
    """

    # initialize contingency table
    crosstab = pd.DataFrame(columns = columns, index = index)
    
    # fill contingency table with values in the subset

    for v in dict_vacORnot: # loop throug vaccinated vs. not

        
        # with AEs

        # number of nonvaccinated/vaccinated people with AEs = number of individuals with AEs
        crosstab.loc[v,'AE'] = len(df_event[df_event['vaccinated']==dict_vacORnot[v]]['eid'].unique())
        
        # DELETE THIS FOR FINAL ANALYSIS
        # if we have no event (happens in the subset), add 1 for testing the plot and analysis further below
        # -> still visible in the output that it was added
        if crosstab.loc[v,'AE']==0:
            crosstab.loc[v,'AE']=1

            
        # without AEs

        if v == 'vaccinated': # for vaccinated people

            # total number of vaccinated individuals - those with AEs
            crosstab.loc[v,'no AE'] = len(id_vac) - crosstab.loc[v,'AE']

        else: # for not vaccinated people
            
            # using id_vac
            crosstab.loc[v,'no AE'] = len(id_nvac) - crosstab.loc[v,'AE']
            
    
    # adding margins to the contingency table (=total value as row and column)
    
    # vaccinated vs. not
    crosstab.loc['Total',:] = np.sum(crosstab, axis = 0)
    # event vs. no event
    crosstab.loc[:,'Total'] = np.sum(crosstab, axis = 1)

    return crosstab
    

In [ ]:
crosstab_all = contingency(df_AEs, id_vac, id_nvac)
crosstab_all

Chi-squared test

In [ ]:
# define significance treshold
alpha = 0.05

# perform chi-squared test:
stat, p, dof, expected = sp.stats.chi2_contingency(crosstab_all.iloc[0:2,0:2], 1)

p

Calculate risk ratio (similar to Barda et al.)

In [ ]:
def riskratio(crosstab, z = 1.96):
    """
    Calculates the risk ratio by adding a row and a column with the total count to the contingency table.
    
    Returns the risk ratio with the confidence interval (CI) as a df and the adjusted contingency table:
    (RR, crosstab)
    
    Important: the input (crosstab) needs to have the following format:
    - 2x2
    - column 0 = events (e.g. AEs), column 1 = non-events
    - row 0 = intervention (e.g. vaccinated), column 1 = non-vaccinated
    """
    
    # calculating the risk ratio

    # risk of getting AE when vaccinated
    R_vac = crosstab.loc['vaccinated','AE']/crosstab.loc['vaccinated','Total']
    # risk of getting "AE" when not vaccinated
    R_nvac = crosstab.loc['not vaccinated', 'AE']/crosstab.loc['not vaccinated', 'Total']

    # risk ratio
    RR = R_vac / R_nvac
    
    # natural log of the RR
    logRR = np.log(RR) 

    # standard error
    se = np.sqrt(crosstab.loc['vaccinated','no AE']/ \
                 (crosstab.loc['vaccinated','AE']*crosstab.loc['vaccinated','Total']) + \
                crosstab.loc['not vaccinated','no AE']/ \
                (crosstab.loc['not vaccinated','AE']*crosstab.loc['not vaccinated','Total']))

    # Assuming normal distribution, 95% lie around 1.96 standard deviations of the mean.
    # as 95% CI is common, the z-score (z) 1.96 is put as the default value.

    # 1-alpha confidence interval (CI) 
    CI_lower = np.exp(np.log(RR) - se * z) # lower bound
    CI_upper = np.exp(np.log(RR) + se * z) # upper bound    
    
    # put the result in a series
    RR_result = pd.Series(data = [RR, CI_lower, CI_upper], index = ['RR', 'CI_lower', 'CI_upper'])
    
    return RR_result

note: se and CI calculated as described on https://en.wikipedia.org/wiki/Relative_risk. 

In [ ]:
RR_all = riskratio(crosstab_all)
RR_all

plot the RR & CI

In [ ]:
plt.figure(figsize = (5,5))
plt.yscale("log")
plt.axhline(1, ls='--', linewidth=1, color='black')
    
# draw the CI
x = [0,0] # x location at 0
y = [RR_all['CI_lower'], RR_all['CI_upper']]
plt.plot(x, y, "_-", markersize = 15, markeredgewidth= 3, linewidth = 3, color = 'b')

# draw the RR
x = [0]
y = [RR_all['RR']]
plt.plot(x, y, "o", markersize = 10, color = 'b')

# labels
plt.xlabel('')
plt.ylabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination in the whole population')
plt.xticks([]) # empty list gives no x-ticks

## 2. What about people with medical history?

Plot incidence

In [ ]:
# get dataset with only AEs AND people with underlying disease
# subset of the df with all AEs (df_AEs) with only people with underlying diseases (df_AEs_ud)
# reminder: leave R as adverse event (and general underlying disease) & Q as underlying disease.

# subset of clinical_vac df with only medical events before
df_ud= clinical_vac[clinical_vac['before_after_vaccine']== 'before']

# drop rows without icd10_code, which correspond to vaccinated people without underlying disease
df_ud.dropna(subset = ['icd10_code'], inplace = True)

# remove R (less well defined condition), which are not considered as underlying disease
# df_ud = df_ud[~df_ud['icd10_code'].str.startswith('R')]

# get id's of people with an underlying disease
id_ud = df_ud['eid'].unique()

# subset of the df with AEs with only people with an underlying disease
df_AEs_ud = df_AEs[df_AEs['eid'].isin(id_ud)]
df_AEs_ud

In [ ]:
# individuals in subset with only people with underlying disease AND the vaccine
id_ud_vac = df_ud[df_ud['vaccinated']==True]['eid'].unique()

# individuals in subset with only people with underlying disease AND NOT the vaccine
id_ud_nvac = df_ud[df_ud['vaccinated']==False]['eid'].unique()

In [ ]:
df_sum_ud = incidence(df_AEs_ud, id_ud_vac, id_ud_nvac)
df_sum_ud

In [ ]:
plt.figure(figsize = (5,5))
plt.bar(df_sum_ud.index, df_sum_ud['Number of AEs per thousand individuals'])
plt.ylabel('Number of AEs per thousand individuals')
plt.title('Adverse events in people with underlying diseases')

Chi-squared test

In [ ]:
# contingency table
crosstab_ud = contingency(df_AEs_ud, id_ud_vac, id_ud_nvac, columns = ls_AE, index = dict_vacORnot)
crosstab_ud

In [ ]:
# define significance treshold
alpha = 0.05

# perform chi-squared test:
stat, p, dof, expected = sp.stats.chi2_contingency(crosstab_ud.iloc[0:2,0:2], 1)

p

Calculate and plot risk ratio

In [ ]:
RR_ud = riskratio(crosstab_ud)
RR_ud

In [ ]:
plt.figure(figsize = (5,5))
plt.yscale("log")
plt.axhline(1, ls='--', linewidth=1, color='black')
    
# draw the CI
x = [0,0] # x location at 0
y = [RR_ud['CI_lower'], RR_ud['CI_upper']]
plt.plot(x, y, "_-", markersize = 15, markeredgewidth= 3, linewidth = 3, color = 'b')

# draw the RR
x = [0]
y = [RR_ud['RR']]
plt.plot(x, y, "o", markersize = 10, color = 'b')

# labels
plt.xlabel('')
plt.ylabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination in people with underlying diseases')
plt.xticks([]) # empty list gives no x-ticks

In [ ]:
# create a df for collecting the results of the risk ratio to compare the populations

# initialize
RR_result = pd.DataFrame(index = ['whole population', 'people with underlying diseases'], columns = ['RR', 'CI_lower', 'CI_upper'])

# fill it
# RR with data from all individuals in the scripts & clinical dataset
RR_result.loc['whole population',:] = RR_all
# RR with data from all individuals with underlying diseases
RR_result.loc['people with underlying diseases',:] = RR_ud

RR_result

In [ ]:
loc = []
loc.append(3)
loc

In [ ]:
# plot the difference

# fig, ax = plt.subplots(figsize=(7, 5))
plt.figure(figsize = (5,5))
plt.yscale("log")
plt.axhline(1, ls='--', linewidth=1, color='black')

color = ['g','b']

# initialize counter
c = 0
# initialize location array for labelling x ticks
loc = []

# looping through the rows, i = index of row and res = data as a series
for i, res in RR_result.iterrows():
    
    # draw the CI
    x = [c,c] # x location
    y = [res['CI_lower'], res['CI_upper']]
    plt.plot(x, y, "_-", markersize = 15, markeredgewidth= 3, linewidth = 3, color = color[c])
    
    # draw the RR
    x = [c] # x location
    y = [res['RR']]
    plt.plot(x, y, "o", markersize = 10, color = color[c])
    
    # append the x location to the location list for labelling x ticks
    loc.append(c)
    c += 1

# labelling
plt.ylabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination in people with underlying diseases or the general population')
plt.xticks(ticks = loc, labels = RR_result.index)
plt.xlim(-0.5,1.5)

## 3. Are they more common in people with (specific) underlying medical conditions?

Incidence

In [ ]:
# dictionary of disease categories to analyse, from https://www.icd10data.com/ICD10CM/Codes
dict_cat = {('A','B'):'Certain infectious and parasitic diseases',
           ('C','D0','D1','D2','D3','D4'):'Neoplasms',
           ('D5','D6','D7','D8'):'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
           'E':'Endocrine, nutritional and metabolic diseases',
           'F':'Mental, Behavioral and Neurodevelopmental disorders',
           'G':'Diseases of the nervous system',
           ('H0','H1','H2','H3','H4','H5'):'Diseases of the eye and adnexa',
           ('H6','H7','H8', 'H9'): 'Diseases of the ear and mastoid process',
           'I':'Diseases of the circulatory system',
           'J':'Diseases of the respiratory system',
           'K':'Diseases of the digestive system',
           'L':'Diseases of the skin and subcutaneous tissue',
           'M':'Diseases of the musculoskeletal system and connective tissue',
           'N':'Diseases of the genitourinary system',
           'Q':'Congenital malformations, deformations and chromosomal abnormalities'}

# initialize dataframe for collecting values with/without vaccinated people for all categories
df_sum_cat_all = pd.DataFrame(columns = dict_vacORnot, index=dict_cat.values()) 

for i, ic in enumerate(dict_cat): # loop over ICD10 codes of each disease category
  
    # get dataset with only AEs AND people with specific underlying disease
    # subset of the df with all AEs (df_AEs) with only people with specific underlying diseases
    # reminder: leave R as adverse event & Q as underlying disease.
    # df_ud = subset of clinical_vac df with only diagnoses before vaccine as defined further above

    # subset with only underlying diseases within the specific category
    df_cat = df_ud[df_ud['icd10_code'].str.startswith(ic)]

    # IDs with underlying disease in the specific category
    id_cat = df_cat['eid'].unique()

    # subset of the df with AEs with only people with underlying disease in the specific category
    df_AEs_cat = df_AEs[df_AEs['eid'].isin(id_cat)]

    # get IDs of individuals in this subset with vs without vaccine 
    id_cat_vac = df_cat[df_cat['vaccinated']==True]['eid'].unique() # with vaccine
    id_cat_nvac = df_cat[df_cat['vaccinated']==False]['eid'].unique() # without vaccine

    df_sum_cat = incidence(df_AEs_cat, id_cat_vac, id_cat_nvac)
    
    # put in the values from the summary table
    df_sum_cat_all.loc[dict_cat[ic],:] = df_sum_cat.loc[:, 'Number of AEs per thousand individuals']
    
df_sum_cat_all.sort_values(by = 'vaccinated', ascending = False, inplace = True) # sorting
df_sum_cat_all

In [ ]:
plt.figure(figsize = (8,8))

# plot results for vaccinated and not on top of each other
for v in dict_vacORnot:
    plt.barh(df_sum_cat_all.index, df_sum_cat_all[v], alpha = 0.5, label = v)

# labelling
plt.title('Adverse events in people with diseases in specific ICD10 categories')
plt.xlabel('Number of AEs per thousand individuals')
plt.legend(loc = "upper right")


Chi-squared test

Risk ratio

# Unused code so far

In [ ]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

In [ ]:
# getting corresponding CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
# code[m_circ_CTV3]

In [ ]:
 # getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask

# correspond to vaccinated or not

In [ ]:
# no one with any vaccine had circulatory issue before vaccine on this limited dataset
scripts[mask_vac]['eid'].isin(clinical[m_circ_sub]['eid']).sum()